In [11]:
import numpy as np
from scipy import *

In [12]:
ka = 10 #W/m°C 
kb = 1 #W/m°C
La = 73 #mm
Lb = 17 #mm
h = 80 ##W/m²°C
T_infinito = 20 #°C
q0 = 4000 #W/m
A = 1 #m²; área transversal hipotética

'''
for k in [ka,kb]:
    for 

'''

meshA = 9
meshB = 5

deltax_A = La/meshA

#para matarial A
k = ka


ae = k*A/deltax_A